In [1]:
import numpy as np
import pandas as pd
import cv2
from sklearn.utils.multiclass import unique_labels
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras import Sequential
from keras.models import load_model
from keras.applications.vgg16 import VGG16 #For Transfer Learning
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD,Adam
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout
from tensorflow.keras.utils import to_categorical

In [2]:
data = pd.read_csv('../input/ct-kidney-dataset-normal-cyst-tumor-and-stone/kidneyData.csv')

In [3]:
data

In [4]:
data.info()

In [5]:
data['Class'].unique()

In [6]:
kidney_classes = data['Class'].value_counts()
kidney_classes

In [7]:
plot_bar = kidney_classes.plot.bar(title='Total number of items in classes')
plot_bar.bar_label(plot_bar.containers[0]);

In [8]:
pip install split_folders

In [9]:
import splitfolders
splitfolders.ratio("../input/ct-kidney-dataset-normal-cyst-tumor-and-stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone", output="dataset", seed=1337, ratio=(.8, .1, .1), group_prefix=None)

In [10]:
channels=3
batch_size=25
test_batch_size=32 
test_steps=1
train_path = './dataset/train'
test_path = './dataset/test'
val_path = './dataset/val'

def scalar(img):    
    return img  # EfficientNet expects pixelsin range 0 to 255 so no scaling is required

train_gen=ImageDataGenerator(preprocessing_function=scalar, horizontal_flip=True)
test_gen=ImageDataGenerator(preprocessing_function=scalar)
train_generator=train_gen.flow_from_directory( directory=train_path , target_size=(224,224), class_mode='categorical',
                                     shuffle=True, batch_size=batch_size)
test_generator=test_gen.flow_from_directory( directory=test_path, target_size=(224,224), class_mode='categorical',
                                     shuffle=False, batch_size=test_batch_size)

valid_generator=test_gen.flow_from_directory( directory=val_path, target_size=(224,224), class_mode='categorical',
                                     shuffle=True, batch_size=batch_size)
classes=list(train_generator.class_indices.keys())
class_count=len(classes)
train_steps=int(np.ceil(len(train_generator.labels)/batch_size))

In [11]:
base_model = VGG16(include_top = False, input_shape = (224,224,3), classes = 38)
model= Sequential()
model.add(base_model) 
model.add(Flatten()) 

In [12]:
model.summary()

In [13]:
model.add(Dense(1024,activation=('relu'),input_dim=512))
model.add(Dense(512,activation=('relu'))) 
model.add(Dense(256,activation=('relu'))) 
model.add(Dropout(.3))
model.add(Dense(128,activation=('relu')))
model.add(Dense(4,activation=('softmax'))) 

#Checking the final model summary
model.summary()

In [14]:
batch_size= 1218
epochs=15
learn_rate=.001
sgd=SGD(learning_rate=learn_rate,momentum=.9,nesterov=False)
adam=Adam(learning_rate=learn_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])

In [15]:
history = model.fit(train_generator,epochs = 10, validation_data = valid_generator, verbose = 1)

In [16]:
def print_info( test_gen, preds, save_dir, subject ):
    class_dict=test_gen.class_indices
    labels= test_gen.labels
    file_names= test_gen.filenames 
    error_list=[]
    true_class=[]
    pred_class=[]
    prob_list=[]
    new_dict={}
    error_indices=[]
    y_pred=[]
    for key,value in class_dict.items():
        new_dict[value]=key             # dictionary {integer of class number: string of class name}
    # store new_dict as a text fine in the save_dir
    classes=list(new_dict.values())     # list of string of class names     
    errors=0      
    for i, p in enumerate(preds):
        pred_index=np.argmax(p)         
        true_index=labels[i]  # labels are integer values
        if pred_index != true_index: # a misclassification has occurred
            error_list.append(file_names[i])
            true_class.append(new_dict[true_index])
            pred_class.append(new_dict[pred_index])
            prob_list.append(p[pred_index])
            error_indices.append(true_index)            
            errors=errors + 1
        y_pred.append(pred_index)    
#     if errors>0:
#         plot_bar=[]
#         plot_class=[]
#         for  key, value in new_dict.items():        
#             count=error_indices.count(key) 
#             if count!=0:
#                 plot_bar.append(count) # list containg how many times a class c had an error
#                 plot_class.append(value)   # stores the class 
#         fig=plt.figure()
#         fig.set_figheight(len(plot_class)/3)
#         fig.set_figwidth(10)
#         plt.style.use('fivethirtyeight')
#         for i in range(0, len(plot_class)):
#             c=plot_class[i]
#             x=plot_bar[i]
#             plt.barh(c, x, )
#             plt.title( ' Errors by Class on Test Set')
    y_true= np.array(labels)        
    y_pred=np.array(y_pred)
    if len(classes)<= 40:
        # create a confusion matrix 
        cm = confusion_matrix(y_true, y_pred )        
        length=len(classes)
        if length<8:
            fig_width=8
            fig_height=8
        else:
            fig_width= int(length * .5)
            fig_height= int(length * .5)
        plt.figure(figsize=(fig_width, fig_height))
        sns.heatmap(cm, annot=True, vmin=0, fmt='g', cmap='Blues', cbar=False)       
        plt.xticks(np.arange(length)+.5, classes, rotation= 90)
        plt.yticks(np.arange(length)+.5, classes, rotation=0)
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title("Confusion Matrix")
        plt.show()
    clr = classification_report(y_true, y_pred, target_names=classes)
    print("Classification Report:\n----------------------\n", clr)

In [17]:
import seaborn as sns
sns.set_style('darkgrid')
from PIL import Image
from sklearn.metrics import confusion_matrix, classification_report
p = model.predict(test_generator)
print_info( test_generator, p, r'./', 'kidney')  

In [18]:
model.evaluate(test_generator)

In [19]:
model.save("vgg16.h5")

In [20]:
import shutil 
shutil.rmtree('./dataset')

In [21]:
history.params
history.history.keys()

In [22]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

In [23]:
loss = history.history['loss']
val_loss = history.history['val_loss']

In [24]:
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(range(10), acc, label='Training Accuracy')
plt.plot(range(10), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

In [25]:
plt.subplot(1, 2, 2)
plt.plot(range(10), loss, label='Training Loss')
plt.plot(range(10), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
path = input("Enter Image path: ")

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

image = cv2.imread(path)

fig = plt.figure(figsize=(100,100))
 
# Use the cvtColor() function to grayscale the image
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
fig.add_subplot(1, 4, 1)
plt.imshow(gray_image,interpolation='nearest')
plt.axis('off')

img_median = cv2.medianBlur(gray_image, 5)

fig.add_subplot(1, 4, 2)
plt.imshow(img_median,interpolation='nearest')
plt.axis('off')
# equ = cv2.equalizeHist(img_median)

gamma_corrected = np.array(255*(img_median / 255) ** 2.5, dtype = 'uint8')

In [ ]:
fig.add_subplot(1, 4, 3)
plt.imshow(gamma_corrected,interpolation='nearest')
plt.axis('off')

ret, thresh1 = cv2.threshold(gamma_corrected, 120, 255, cv2.THRESH_BINARY)

fig.add_subplot(1, 4, 4)
plt.imshow(thresh1,interpolation='nearest')
plt.axis('off')